# Tools and Routing


- Functions and services can LLM can utilize to extend its capabilities are named "tools" in LangChain
- LangChain has many tools available
  - Search tools
  - Math tools
  - SQL tools
    ...

In this notebook, we will:

- Create our own tools
- Build a tool based on an OpenAPI spec
  - Predating LLMs, the OpenAPI specification is routinely used by service providers to describe their APIs.
- Select from multiple possible tools - called `routing`


In [ ]:
import os
import openai
import json

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain.agents import tool

`tool` is a decorator used to define a function as a tool.


In [ ]:
@tool
def search(query: str) -> str:
    """Searches for weather online"""
    return "21c"


print(search)

In [ ]:
search.name

In [ ]:
search.description

In [ ]:
search.args

To improve upon the `tool` decorator, by defining a more explicit structure for the input schema.

This is important because the description of the input is what the LLM uses to determine what the input should be.

So having a really clear definition of the input becomes important.


In [ ]:
from pydantic import BaseModel, Field


class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

`query` on `SearchInput` matches the parameter of the `search` function.

The main difference is that we're adding a `description` to the `query` parameter.

So passing in a `SearchInput` object to the `args_schema` parameter of the `tool` decorator, will ensure that the LLM knows what the input should be.


In [ ]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Searches for weather online"""
    return "21c"

In [ ]:
search.args

In [ ]:
search.run("sf")

In [ ]:
from tools.weather import get_current_temperature

In [ ]:
get_current_temperature.name

In [ ]:
get_current_temperature.description

In [ ]:
get_current_temperature.args

We can convert this tool to an OpenAI function definition with `format_tool_to_openai_function`.


In [ ]:
from langchain.tools.render import format_tool_to_openai_function

In [ ]:
format_tool_to_openai_function(get_current_temperature)

In [ ]:
get_current_temperature({"latitude": -28, "longitude": -48})

In [ ]:
from tools.wikipedia import search_wikipedia

In [ ]:
search_wikipedia.name

In [ ]:
search_wikipedia.description

In [ ]:
format_tool_to_openai_function(search_wikipedia)

In [ ]:
search_wikipedia({"query": "LangChain"})

### Interacting with OpenAPI specs


In [ ]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [ ]:
text = """
{
  "openapi": "3.1.0",
  "info": {
    "title": "Swagger Petstore",
    "contact": {},
    "version": "1.0.0"
  },
  "jsonSchemaDialect": "https://json-schema.org/draft/2020-12/schema",
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1",
      "variables": {}
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "tags": [
          "pets"
        ],
        "summary": "listPets",
        "description": "",
        "operationId": "listPets",
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "style": "form",
            "explode": true,
            "schema": {
              "maximum": 100.0,
              "type": "integer",
              "contentEncoding": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "content": {
                  "text/plain": {
                    "schema": {
                      "type": "string",
                      "description": "A link to the next page of responses",
                      "contentMediaType": "text/plain"
                    }
                  }
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "maxItems": 100,
                  "type": "array",
                  "items": {
                    "$ref": "#/components/schemas/Pet"
                  },
                  "description": "",
                  "contentMediaType": "application/json"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "headers": {},
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        },
        "deprecated": false
      },
      "post": {
        "tags": [
          "pets"
        ],
        "summary": "createPets",
        "description": "",
        "operationId": "createPets",
        "parameters": [],
        "responses": {
          "201": {
            "description": "Null response",
            "headers": {},
            "content": {}
          },
          "default": {
            "description": "unexpected error",
            "headers": {},
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        },
        "deprecated": false
      },
      "parameters": []
    },
    "/pets/{petId}": {
      "get": {
        "tags": [
          "pets"
        ],
        "summary": "showPetById",
        "description": "",
        "operationId": "showPetById",
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "description": "The id of the pet to retrieve",
            "required": true,
            "style": "simple",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "headers": {},
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "headers": {},
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        },
        "deprecated": false
      },
      "parameters": []
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "title": "Pet",
        "required": [
          "id",
          "name"
        ],
        "type": "object",
        "properties": {
          "id": {
            "type": "integer",
            "contentEncoding": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Error": {
        "title": "Error",
        "required": [
          "code",
          "message"
        ],
        "type": "object",
        "properties": {
          "code": {
            "type": "integer",
            "contentEncoding": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  },
  "tags": [
    {
      "name": "pets",
      "description": ""
    }
  ]
}
"""

In [ ]:
spec = OpenAPISpec.from_text(text)

In [ ]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [ ]:
pet_openai_functions

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [ ]:
model.invoke("what are three pets names")

In [ ]:
model.invoke("tell me about pet with id 42")

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

We're going to make the model choose between two different tools and what inputs should be passed into those paths.


In [ ]:
functions = [
    format_tool_to_openai_function(f)
    for f in [
        search_wikipedia,
        get_current_temperature,
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [ ]:
model.invoke("what is the weather in san francisco")

In [ ]:
model.invoke("what is LangChain?")

### Customizing the prompt

Now we add a a `prompt` before the LLM call.


In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful but sassy assistant"),
        ("user", "{input}"),
    ]
)
chain = prompt | model

There are two main possible **end states** of the LLM's response:

1. When it decides to call a tool;
   > On this first state, we are interested the the `tool` it decides to call as well as the `input` to that tool.
   >
   > For the `input`, we parsed it with `OpenAIFunctionsAgentOutputParser` into a Python dict.
2. When it doesn't decide to call a tool.
   > On this second state, we are interested on the value of `AIMessage.content`


In [ ]:
chain.invoke({"input": "what is the weather in sf right now"})

The `OpenAIFunctionsAgentOutputParser` is a parser that takes the output of the LLM and parses it into a Python dict.

This parser verifies if the output is a:

1. Function call or a response.
2. If it's a function call, what is the input


In [ ]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [ ]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [ ]:
type(result)

In [ ]:
result.tool

In [ ]:
result.tool_input

After it's parsed, we can pass the `tool_input` to the parameter of a tool function and get the output of that tool. 🤯


In [ ]:
get_current_temperature(result.tool_input)

Now an input with no function to call


In [ ]:
result = chain.invoke({"input": "hi!"})

In [ ]:
type(result)

In [ ]:
result.return_values

In [ ]:
from langchain.schema.agent import AgentFinish


def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values["output"]
    else:
        tools = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [ ]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [ ]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [ ]:
result

In [ ]:
result = chain.invoke({"input": "What is LangChain?"})

In [ ]:
result

In [ ]:
chain.invoke({"input": "hi!"})

### Exercise: try to create a tool that multiplies the value for 2


In [94]:
class NumberInput(BaseModel):
    number: int = Field(description="Number to square")

In [100]:
@tool(args_schema=NumberInput)
def square(number: int) -> int:
    """Squares a number"""
    number = int(number)
    return number**2

In [101]:
functions = [
    format_tool_to_openai_function(f)
    for f in [
        search_wikipedia,
        get_current_temperature,
        square,
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)


def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values["output"]
    else:
        tools = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
            "square": square,
        }
        return tools[result.tool].run(result.tool_input)


chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [104]:
chain.invoke({"input": "What is 6 squared?"})

36